In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import pickle
import random

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/resume-data-with-annotations/resume v6.pdf
/kaggle/input/resume-data-with-annotations/Alice Clark CV.docx
/kaggle/input/resume-data-with-annotations/Alice Clark CV.txt
/kaggle/input/resume-data-with-annotations/AshlyLauResume.pdf
/kaggle/input/resume-data-with-annotations/Smith Resume.pdf
/kaggle/input/resume-data-with-annotations/train_data.txt
/kaggle/input/resume-data-with-annotations/Smith Resume.docx
/kaggle/input/resume-data-with-annotations/Jinesh_Dhruv_poster.pdf
/kaggle/input/resume-data-with-annotations/Alice Clark CV.pdf
/kaggle/input/resume-data-with-annotations/train_data.pkl
/kaggle/input/resume-data-with-annotations/Sample_Resume.pdf
/kaggle/input/resume-data-with-annotations/Xinni_Chng.pdf
/kaggle/input/resume-data-with-annotations/sample_input.pdf


In [ ]:
train_data = pickle.load(open('/kaggle/input/resume-data-with-annotations/train_data.pkl','rb'))
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [5]:
# Load Blank Model
nlp = spacy.blank('en')

def train_model(train_data):
    # Remove all pipelines and add NER pipeline from the model
    if 'ner'not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        # adding NER pipeline to nlp model
        nlp.add_pipe(ner,last=True)

    #Add labels in the NLP pipeline
    for _, annotation in train_data:
        for ent in annotation.get('entities'):
            ner.add_label(ent[2])

    #Remove other pipelines if they are there
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10): # train for 10 iterations
            print("Starting iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass

            print(losses)
# Start Training model
train_model(train_data)

Starting iteration 0
{'ner': 11880.518611802248}
Starting iteration 1
{'ner': 9549.533454774772}
Starting iteration 2
{'ner': 6647.957126085104}
Starting iteration 3
{'ner': 7000.408218887242}
Starting iteration 4
{'ner': 5665.557018652162}
Starting iteration 5
{'ner': 6860.215155211636}
Starting iteration 6
{'ner': 5207.470807416294}
Starting iteration 7
{'ner': 4684.51306761087}
Starting iteration 8
{'ner': 5968.817534300315}
Starting iteration 9
{'ner': 4254.501001652606}


In [ ]:
# Saving the model
nlp.to_disk('nlp_ner_model')

In [ ]:
#Loading Model
nlp_model = spacy.load('nlp_ner_model')

In [ ]:
# trying and seeing the prediction of the model
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f"{ent.label_.upper():{30}}-{ent.text}")

NAME                          -amarjyot sodhi
DESIGNATION                   -Voice and Accent Trainer
COMPANIES WORKED AT           -3 years
LOCATION                      -Faridabad
EMAIL ADDRESS                 -indeed.com/r/amarjyot-sodhi/ba2e5a3cbaeccdac
DESIGNATION                   -Sales Associate
COMPANIES WORKED AT           -Shuttl
LOCATION                      -Faridabad
LOCATION                      -Chennai
DEGREE                        -masters in journalism and communication in mass comm client servicing
COLLEGE NAME                  -amity -  Noida


In [1]:
!pip install PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 29.8 MB/s eta 0:00:00


In [3]:
import sys, fitz
fname = '/kaggle/input/resume-data-with-annotations/Alice Clark CV.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text = text + str(page.getText())
tx = " ".join(text.split('\n'))
print(tx)

Alice Clark  AI / Machine Learning    Delhi, India Email me on Indeed  •  20+ years of experience in data handling, design, and development  •  Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to  data warehousing and business intelligence  •  Database: Experience in database designing, scalability, back-up and recovery, writing and  optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes.  Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure,  Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake  analytics(U-SQL)  Willing to relocate anywhere    WORK EXPERIENCE  Software Engineer  Microsoft – Bangalore, Karnataka  January 2000 to Present  1. Microsoft Rewards Live dashboards:  Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping  online. Microsoft Rewards members can earn points when searching with Bing, bro

In [4]:
# Applying the model
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                           - Alice Clark
DESIGNATION                    - Software Engineer
LOCATION                       - Bangalore
COMPANIES WORKED AT            - Microsoft
GRADUATION YEAR                - 2000
COLLEGE NAME                   - Indian Institute of Technology
SKILLS                         - Machine Learning, Natural Language Processing, and Big Data Handling


In [ ]:
# Applying the model, it need not be a pdf but also text!!
tx = "John Smith is a Software Engineer with expertise in Python and Java. He can be reached at john.smith@example.com."
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')



NAME                          - John Doe
JOB_TITLE                     - Senior Data Scientist
COMPANY                       - XYZ Analytics
LOCATION                      - New York City
DEGREE                        - Master's degree in Computer Science
COLLEGE_NAME                  - ABC University
EMAIL                         - john.doe@example.com
PHONE                         - 123-456-7890

